### Creating an instance

In [3]:
nodes = [i for i in range(1, 11)] # N

coords = [[20355.966023, 16167.127237], [39988.592020, 19773.197847], [23572.548971, 31529.184022], 
[37304.369590, 32079.249435], [24512.513179, 38835.826089], [40867.253869, 38565.884488], 
[27520.840478, 46921.515986], [36067.877874, 44894.490748], [19345.710923, 51972.040940], [33635.749233, 49663.479404]]

distance_matrix = np.zeros((10, 10))

def euclidean_dist(a, b):
    return np.sqrt((b[0] - a[0]) ** 2 + (b[1] - a[1]) ** 2)

for i in nodes:
    for j in nodes:
        distance_matrix[i - 1][j - 1] = euclidean_dist(coords[i - 1], coords[j - 1])

In [4]:
potential_hubs = [i for i in nodes] # H

In [5]:
capacity_levels = [1, 2, 3] # Q

In [6]:
commodities = [] # set K

for i in nodes:
    for j in nodes:
        k = (i - 1) * 10 + j
        commodities.append(k)

In [7]:
demand = np.array([[75.455160, 36.992250, 54.503740, 19.269930, 20.093530, 17.650830, 56.770230, 17.080500, 18.831190, 16.385700],
                   [25.793680, 38.375560, 24.997800, 26.576220, 18.289200, 23.168780, 38.161870, 20.412130, 12.482780, 16.597520],
                   [66.812590, 39.315400, 51.186310, 22.408500, 24.202840, 20.514430, 70.558250, 20.242000, 22.999690, 19.424020],
                   [17.329130, 33.119610, 18.761160, 24.907980, 16.361600, 22.929530, 40.161070, 23.146280, 11.420450, 18.670240],
                   [17.485420, 25.137020, 21.182770, 19.093040, 23.480380, 18.306310, 73.409770, 20.087670, 23.889580, 19.434310],
                   [11.607170, 18.441950, 12.168340, 16.115400, 11.117000, 16.942850, 37.166850, 21.828940, 9.556350, 18.066350],
                   [82.372600, 77.535140, 90.449230, 67.791970, 95.057940, 72.635600, 312.258790, 98.826690, 110.006660, 110.368410],
                   [35.143120, 41.734510, 36.299460, 48.415280, 35.711070, 57.578140, 173.373140, 89.946000, 41.226350, 91.825380],
                   [12.196230, 11.899530, 15.345270, 10.239040, 18.935880, 11.158610, 67.875160, 15.064250, 23.622110, 20.372560],
                   [21.717650, 24.963930, 22.806940, 27.790220, 23.083630, 32.891690, 109.098310, 50.846560, 30.247210, 63.327840]])

demand_flattened = demand.flatten() # W_k vector

In [8]:
# for bookkeeping
commodities_with_routing_info = {}

for i in nodes:
    for j in nodes:
        k = (i - 1) * 10 + j
        commodities_with_routing_info[k] = [i, j, demand_flattened[k - 1]]

In [9]:
hub_capacities = [2198.532635, 6106.711881, 2226.593104, 4515.466789, 4423.706323, 5748.026876, 6109.336283, 6425.976443, 4740.035887, 6238.554903]
fixed_setup_costs = [28766.736921, 28376.761527, 29774.238965, 24301.334212, 25853.461856, 20762.874300, 34166.355726, 33859.209322, 24717.891218, 33686.431972]

new_hub_capacities = {} # b_iq
fixed_set_up_costs_with_capacity_level = {} # f_iq

for i in reversed(potential_hubs):
    for q in reversed(capacity_levels):

        if q == 3:
            new_hub_capacities[(i, q)] = hub_capacities[i - 1]
            fixed_set_up_costs_with_capacity_level[(i, q)] = fixed_setup_costs[i - 1]

        elif q == 2:
            new_hub_capacities[(i, q)] = 0.7 * hub_capacities[i - 1]
            fixed_set_up_costs_with_capacity_level[(i, q)] = 0.9 * fixed_setup_costs[i - 1]

        else:
            new_hub_capacities[(i, q)] = 0.7 ** 2 * hub_capacities[i - 1]
            fixed_set_up_costs_with_capacity_level[(i, q)] = 0.9 ** 2 * fixed_setup_costs[i - 1]


In [10]:
# F_ijk values
def distance_between_nodes(i, j):
    return euclidean_dist(distance_matrix[i - 1], distance_matrix[j - 1])

collection_cost = 3.000000
transfer_cost = 0.750000
distribution_cost = 2.000000

def cost_of_route(i, j, k):
    return demand_flattened[k - 1] * (collection_cost * distance_between_nodes(origin(k), i) + transfer_cost * distance_between_nodes(i, j) + distribution_cost * distance_between_nodes(j, destination(k)))

def origin(k):
    return commodities_with_routing_info[k][0]

def destination(k):
    return commodities_with_routing_info[k][1]